In [ ]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-

import os
import glob
import time
import pandas as pd
from docx import Document
from transformers import pipeline
from huggingface_hub import login

# =====================================
#  設定
# =====================================
login(token="")
MODEL_ID = "Qwen/Qwen2.5-1.5B-Instruct"

# 大学サーバーでの対象フォルダ
FOLDER = "/home/k101okab/k_data/jiiken5/5-3/PBL/tv"


# =====================================
#  DOCX 読み込み
# =====================================
def read_docx(path):
    try:
        doc = Document(path)
        return "\n".join([p.text.strip() for p in doc.paragraphs if p.text.strip()])
    except Exception as e:
        print(f"❌ 読み込みエラー: {path} → {e}")
        return ""


# =====================================
#  要約関数（100字前後で自然にまとめる）
# =====================================
def summarize_local(text: str, target_chars: int = 100):

    generator = pipeline(
        "text-generation",
        model=MODEL_ID,
        device_map="auto",
        torch_dtype="auto"
    )

    # 100字前後 → 大体 50〜80 tokens 必要
    max_new_tokens = 120

    prompt = (
        f"あなたは優秀な日本語の要約アシスタントです。\n"
        f"以下の文章を *{target_chars}字前後自然に要約してください。\n"
        "文章を途中で切らず、文として自然に終わるようにしてください。\n"
        "要約は一つだけ生成してください。\n\n"
        f"【入力】\n{text}\n\n【要約】"
    )

    result = generator(
        prompt,
        max_new_tokens=max_new_tokens,
        temperature=0.3,
        top_p=0.9,
        do_sample=True,
        num_return_sequences=1,
        pad_token_id=generator.tokenizer.eos_token_id
    )[0]["generated_text"]

    summary = result.replace(prompt, "").strip()

    # 不要な改行削除
    lines = [ln.strip() for ln in summary.split("\n") if ln.strip()]
    summary = " ".join(lines)

    # 文章の途中切れ防止 → 最後の「。」までで切る
    last_period = summary.rfind("。")
    if last_period != -1:
        summary = summary[:last_period + 1]

    # もし短すぎることがあればそのまま返す
    return summary, len(summary)


# =====================================
#  フォルダ一括処理
# =====================================
def main():

    # 文字数（デフォルト100字）
    try:
        char_limit = int(input("要約の文字数を指定してください（例: 100）: "))
    except ValueError:
        char_limit = 100
        print("⚠ 無効のため 100字で実行します")

    paths = glob.glob(os.path.join(FOLDER, "*.docx"))
    if not paths:
        print("⚠ .docx ファイルが見つかりません")
        return

    results = []

    print(f"\n📄 一括処理を開始します…（{len(paths)} ファイル）\n")

    for idx, path in enumerate(paths, 1):
        fname = os.path.basename(path)
        print(f"[{idx}/{len(paths)}] {fname} 要約中…")

        text = read_docx(path)
        if not text:
            continue

        summary, length = summarize_local(text, target_chars=char_limit)

        results.append({
            "ファイル名": fname,
            "本文冒頭100文字": text[:100],
            "要約": summary,
            "要約文字数": length
        })

    # CSV 出力
    timestamp = time.strftime("%Y%m%d_%H%M%S")
    out_csv = os.path.join(FOLDER, f"summary_{timestamp}.csv")

    pd.DataFrame(results).to_csv(out_csv, encoding="utf-8-sig", index=False)
    print(f"\n完了！ CSV を出力しました → {out_csv}")


# =====================================
#  実行
# =====================================
if __name__ == "__main__":
    main()


要約の文字数を指定してください（例: 100）:  100



📄 一括処理を開始します…（74 ファイル）

[1/74] 210829.docx 要約中…


Device set to use cuda:0


[2/74] 220501.docx 要約中…


Device set to use cuda:0


...

[73/74] 230820-原稿３.docx 要約中…


Device set to use cuda:0


[74/74] 231105-鈍川NA_兵動.docx 要約中…


Device set to use cuda:0



🎉 完了！ CSV を出力しました → /home/k101okab/k_data/jiiken5/5-3/PBL/tv/summary_20251212_005244.csv
